In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import gspread
from gspread_dataframe import set_with_dataframe
import time

In [2]:
# Symbols to process
symbols = ["XAUUSD", "BTCUSD", "BTCEUR", "ETHUSD", "SOLUSD", "XRPUSD", 
           "DSHUSD", "XAUEUR", "GBPAUD", "GBPNZD", "CHFJPY", 
           "EURJPY", "USA30", "USA100", "USOIL", "UKOIL", "USDJPY", 
           "GBPJPY", "USA500", "UK100", "CADJPY", "AUDJPY", "GBPCHF", 
           "GBPCAD", "NZDJPY", "GBPUSD", "EURGBP"]

# Initialize MetaTrader 5
if not mt5.initialize():
    print(f"MetaTrader 5 initialization failed with error code {mt5.last_error()}")
    exit()

# Function to find the last available trading day
def get_last_trading_day(reference_date, days_back):
    while days_back > 0:
        reference_date -= timedelta(days=1)
        if reference_date.weekday() in [5, 6]:  # Skip Saturday (5) and Sunday (6)
            continue
        days_back -= 1
    return reference_date

# Function to fetch trading data
def fetch_trading_data(symbols):
    results = []
    now = datetime.now()
    weekday = now.weekday()

    # Assign the correct trading days
    if weekday == 0:  # Monday
        yesterday = get_last_trading_day(now, 3)  # Friday
        day_before_yesterday = get_last_trading_day(now, 4)  # Thursday
    elif weekday == 1:  # Tuesday
        yesterday = get_last_trading_day(now, 1)  # Monday
        day_before_yesterday = get_last_trading_day(now, 4)  # Friday
    else:  # Wednesday - Friday
        yesterday = get_last_trading_day(now, 1)
        day_before_yesterday = get_last_trading_day(now, 2)

    days = [("C", now), ("Y", yesterday), ("V", day_before_yesterday)]

    for symbol in symbols:
        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} is not available.")
            continue

        for level, date in days:
            start = datetime(date.year, date.month, date.day, 0, 0)
            end = start + timedelta(days=1)

            print(f"[{datetime.now()}] Fetching {level} data for {symbol} from {start.date()}")

            rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, start, end)
            
            if rates is not None and len(rates) > 0:
                high = max(rate['high'] for rate in rates)
                low = min(rate['low'] for rate in rates)
                results.append({"Date": start.date(), "Symbol": symbol, "Highest": high, "Lowest": low, "Level": level})
            else:
                print(f"No data found for {symbol} on {start.date()}, using previous data")
                last_trading_day = get_last_trading_day(date, 1)
                last_start = datetime(last_trading_day.year, last_trading_day.month, last_trading_day.day, 0, 0)
                last_end = last_start + timedelta(days=1)
                rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, last_start, last_end)

                if rates is not None and len(rates) > 0:
                    high = max(rate['high'] for rate in rates)
                    low = min(rate['low'] for rate in rates)
                    results.append({"Date": last_start.date(), "Symbol": symbol, "Highest": high, "Lowest": low, "Level": level})
                else:
                    results.append({"Date": last_start.date(), "Symbol": symbol, "Highest": None, "Lowest": None, "Level": level})

    return pd.DataFrame(results)

# Function to upload data to Google Sheets
def upload_to_google_sheets(data, json_path, sheet_name):
    try:
        gc = gspread.service_account(filename=json_path)
        sh = gc.open(sheet_name)
        worksheet = sh.sheet1

        worksheet.clear()
        set_with_dataframe(worksheet, data)

        print(f"[{datetime.now()}] Data successfully uploaded to Google Sheets: {sh.url}")
    except Exception as e:
        print(f"[{datetime.now()}] Error uploading data to Google Sheets: {e}")

# Main function with continuous updates
def main():
    json_path = "C:/Users/Shaheera/Downloads/live-data-449002-016cfa9e813e.json"
    sheet_name = "live_data_updated"

    print("Starting continuous data fetch and upload (Press Ctrl+C to stop)...")
    
    try:
        while True:
            print(f"\n[{datetime.now()}] Fetching new data...")

            trading_data = fetch_trading_data(symbols)
            upload_to_google_sheets(trading_data, json_path, sheet_name)

            print(f"[{datetime.now()}] Next update in 60 seconds...")
            time.sleep(60)  # Wait for 1 minute
    except KeyboardInterrupt:
        print("\nProcess stopped by user.")
    except Exception as e:
        print(f"[{datetime.now()}] Error during execution: {e}")
    finally:
        print("Shutting down MetaTrader 5 connection...")
        mt5.shutdown()

In [ ]:
if __name__ == "__main__":
    main()

Starting continuous data fetch and upload (Press Ctrl+C to stop)...

[2025-02-03 10:56:30.217028] Fetching new data...
[2025-02-03 10:56:30.217028] Fetching C data for XAUUSD from 2025-02-03
[2025-02-03 10:56:30.224742] Fetching Y data for XAUUSD from 2025-01-29
[2025-02-03 10:56:30.224742] Fetching V data for XAUUSD from 2025-01-28
[2025-02-03 10:56:30.225744] Fetching C data for BTCUSD from 2025-02-03
[2025-02-03 10:56:30.239877] Fetching Y data for BTCUSD from 2025-01-29
[2025-02-03 10:56:30.239877] Fetching V data for BTCUSD from 2025-01-28
[2025-02-03 10:56:30.239877] Fetching C data for BTCEUR from 2025-02-03
[2025-02-03 10:56:30.255141] Fetching Y data for BTCEUR from 2025-01-29
[2025-02-03 10:56:30.255141] Fetching V data for BTCEUR from 2025-01-28
[2025-02-03 10:56:30.255141] Fetching C data for ETHUSD from 2025-02-03
[2025-02-03 10:56:30.270755] Fetching Y data for ETHUSD from 2025-01-29
[2025-02-03 10:56:30.270755] Fetching V data for ETHUSD from 2025-01-28
Symbol SOLUSD is 